In [130]:
import scanpy as sc
import scvelo as scv
from utils import add_annotations, compute_nn_from_connectivity

In [131]:
!ls

__pycache__		  developer.ipynb	   training.ipynb
adata_with_latent.h5ad	  gene_sets		   training.py
adata_with_velocity.h5ad  input_processed_anndata  training_pancreas_200.ipynb
data			  model.py		   utils.py
dataloader.ipynb	  modules.py		   vae_velocity_model.pt
dataloader.py		  preprocessing.ipynb


/home/lgolinelli/miniconda3/envs/scvi/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=2148006) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [132]:
adata = scv.datasets.pancreas()

In [133]:
sc.pp.pca(adata, n_comps=100)
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000, subset_highly_variable=True, log=True)
scv.pp.neighbors(adata, n_neighbors=200)
scv.pp.moments(adata, n_neighbors=200)

from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=20 + 1, metric='euclidean', n_jobs=-1)
nbrs.fit(adata.obsm['X_pca'])
distances, indices = nbrs.kneighbors(adata.obsm['X_pca'])

adata.uns['indices'] = indices

Filtered out 20801 genes that are detected 20 counts (shared).
Normalized count data: X, spliced, unspliced.
Extracted 2000 highly variable genes.
Logarithmized X.
computing neighbors


/home/lgolinelli/miniconda3/envs/scvi/lib/python3.13/site-packages/scvelo/preprocessing/utils.py:705: DeprecationWarning: `log1p` is deprecated since scVelo v0.3.0 and will be removed in a future version. Please use `log1p` from `scanpy.pp` instead.
  log1p(adata)
/tmp/ipykernel_2148006/3504472723.py:3: DeprecationWarning: `neighbors` is deprecated since scvelo==0.4.0 and will be removed in a future version of scVelo. Please compute neighbors with Scanpy.
  scv.pp.neighbors(adata, n_neighbors=200)


    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:01) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


In [134]:
adata

AnnData object with n_obs × n_vars = 3696 × 2000
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    var: 'highly_variable_genes', 'gene_count_corr', 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'clusters_coarse_colors', 'clusters_colors', 'day_colors', 'neighbors', 'pca', 'log1p', 'indices'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'spliced', 'unspliced', 'Ms', 'Mu'
    obsp: 'distances', 'connectivities'

In [135]:
add_annotations(
    adata, 
    files=['/home/lgolinelli/git/lineageVI/gene_sets/all_sources_msigdb.gmt'],
    min_genes=12,
    varm_key='I',
    uns_key='terms',
    clean=True,
    genes_use_upper=True)

adata._inplace_subset_var(adata.varm['I'].sum(1) > 0)

In [136]:
adata.obsm['X_pca'].shape

(3696, 100)

In [137]:
adata.write_h5ad('/home/lgolinelli/git/lineageVI/input_processed_anndata/pancreas_PCA_on_Ms_200moments.h5ad')